In [ ]:
# This notebook was used to create a script for execution on the Tufts High Performance Computing Cluster.
# The script takes exported Google Earth engine data and fits a 4-th order Harmonic Regression using SciPy 
# in a pixelwise manner and saves off the coefficients and RMSE as .csv for use in a random forest classifier.

In [1]:
# Import packages
import pandas as pd
import numpy as np
import scipy
import geopandas as gpd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pyogrio
from sklearn.metrics import mean_squared_error

In [2]:
# Working directory
cd "C:/Users/Chad/Desktop/CRWA Project/Working_Data"

C:\Users\Chad\Desktop\CRWA Project\Working_Data


In [3]:
# Read in cleaned data
mygdf=gpd.read_file('2023BandsIndicesCleaned.shp',engine="pyogrio").dropna()

In [4]:
# Simple harmonic function
def harmonicfunc(mult,one,two,x):
    w=2*np.pi/365.25
    return one*np.cos(mult*w*x)+two*np.sin(mult*w*x)

In [5]:
# Complex 4-th order harmonic objective function to fit
def myobj(x,a,b,c,d,e,f,g,h,i):  
    return a+harmonicfunc(1,b,c,x)+harmonicfunc(2,d,e,x)+harmonicfunc(3,f,g,x)+harmonicfunc(4,h,i,x)

In [6]:
# Curve fit for a specific point in a specific band or vegetation index
def curvefit(PtID,band):
    x=mygdf[mygdf['PtID']==PtID]['DOY']
    y=mygdf[mygdf['PtID']==PtID][band]
    popt,pcov=scipy.optimize.curve_fit(myobj, x,y)
    yhat=myobj(x,popt[0],popt[1],popt[2],popt[3],popt[4],popt[5],popt[6],popt[7],popt[8])
    RMSE=np.sqrt(mean_squared_error(y, yhat))
    return RMSE,popt

In [ ]:
# Run curve fitting on each point and in each band/index. Save each as a file with the coefficients and RMSE for each point
output_df=mygdf[['PtID']]
for band in ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12','NDVI','NBR','SAVI','RENDVI','EVI']:
    RMSE_res=np.zeros((output_df.size,1))
    popt_res=np.zeros((output_df.size,9))
    for i in mygdf['PtID'].drop_duplicates():
        RMSE_res[i,:],popt_res[i,:]=curvefit(i,band)
    popt_res=popt_res.T.tolist()
    print(band)
    d={'PtID':mygdf['PtID'].tolist(),'RMSE':RMSE_res.tolist(),'a':popt_res[0],'b':popt_res[1],'c':popt_res[2],'d':popt_res[3],'e':popt_res[4],
       'f':popt_res[5],'g':popt_res[6],'h':popt_res[7],'i':popt_res[8]}
    out=pd.DataFrame(d)
    out.to_csv(band+'harmreg.csv')